# Evaluate Predetermined Top-K Acquisitions
> Resistance is futile.

In [ ]:
# default_exp predetermined_acquisitions_20

In [ ]:
# hide
import blackhc.project.script

Import modules and functions were are going to use.

In [ ]:
# exports

import dataclasses
import traceback
from dataclasses import dataclass
from typing import Optional, Type, Union

import torch
import torch.utils.data
from blackhc.project import is_run_from_ipython
from blackhc.project.experiment import embedded_experiments

from batchbald_redux import acquisition_functions, baseline_acquisition_functions
from batchbald_redux.acquisition_functions import (
    CandidateBatchComputer,
    EvalDatasetBatchComputer,
    EvalModelBatchComputer,
)
from batchbald_redux.batchbald import compute_entropy, get_bald_scores
from batchbald_redux.black_box_model_training import evaluate
from batchbald_redux.dataset_challenges import get_base_dataset_index, get_target
from batchbald_redux.di import DependencyInjection
from batchbald_redux.experiment_data import (
    ExperimentData,
    ExperimentDataConfig,
    OoDDatasetConfig,
    StandardExperimentDataConfig,
)
from batchbald_redux.models import MnistModelTrainer
from batchbald_redux.resnet_models import Cifar10ModelTrainer
from batchbald_redux.train_eval_model import (
    TrainEvalModel,
    TrainSelfDistillationEvalModel,
)
from batchbald_redux.trained_model import BayesianEnsembleModelTrainer, ModelTrainer

In [ ]:
# exports

predetermind_acquisition_base_indices = [
    2271,
    46353,
    50827,
    7715,
    55207,
    14505,
    41730,
    8689,
    54977,
    53221,
    48649,
    16093,
    26191,
    53463,
    14807,
    13293,
    40896,
    18983,
    28380,
    40216,
    38548,
    2767,
    8535,
    17272,
    18852,
    32109,
    17703,
    29678,
    7701,
    45562,
    37381,
    19631,
    4964,
    12015,
    18654,
    9911,
    8020,
    11231,
    8904,
    41934,
    18695,
    7907,
    31009,
    45520,
    104,
    34968,
    19255,
    35136,
    5331,
    13946,
    40844,
    40317,
    14001,
    50898,
    41482,
    42735,
    41532,
    24751,
    19463,
    27399,
    35547,
    42507,
    43223,
    48899,
    39003,
    18493,
    11289,
    46702,
    20101,
    7818,
    3264,
    960,
    7694,
    28584,
    42518,
    28110,
    9179,
    14413,
    45304,
    38286,
    53545,
    53722,
    29454,
    15391,
    198,
    22345,
    36227,
    15551,
    45104,
    47898,
    34830,
    12517,
    20385,
    1454,
    14476,
    33779,
    2710,
    603,
    9859,
    47002,
    55571,
    51653,
    9819,
    13388,
    20566,
    15247,
    35373,
    25575,
    15522,
    4248,
    29553,
    921,
    46304,
    50925,
    13120,
    54484,
    38660,
    27310,
    29133,
    39530,
    22628,
    24596,
    49389,
    39748,
    33066,
    24348,
    29616,
    36733,
    30545,
    8522,
    5118,
    17252,
    10023,
    24159,
    50837,
    45709,
    22520,
    16864,
    19294,
    27534,
    17549,
    52587,
    8557,
    55891,
    46164,
    10439,
    9238,
    49652,
    42634,
    37706,
    30282,
    20850,
    44628,
    36088,
    38228,
    49206,
    46857,
    2166,
    25547,
    4562,
    20126,
    48075,
    22298,
    34407,
    33804,
    28289,
    28189,
    44438,
    47260,
    17907,
    18426,
    44122,
    50942,
    53636,
    49005,
    47026,
    6786,
    40505,
    9570,
    1816,
    55586,
    105,
    50655,
    42207,
    14667,
    42971,
    25429,
    52172,
    48679,
    5113,
    53558,
    11847,
    29984,
    17157,
    2558,
    1268,
    1659,
    26397,
    3958,
    4815,
    30932,
    37512,
    25188,
    31583,
    25434,
    35272,
    2829,
    9548,
    32070,
    11594,
    19424,
    14141,
    21587,
    50174,
    2659,
    17830,
    26256,
    6965,
    37168,
    13853,
    51965,
    6373,
    15030,
    33710,
    15675,
    47745,
    43412,
    32122,
    38180,
    39734,
    27713,
    52331,
    29449,
    13136,
    45188,
    5807,
    49986,
    3096,
    6265,
    12674,
    10763,
    5458,
    18807,
    45448,
    30481,
    26434,
    28192,
    9035,
    40612,
    18839,
    15321,
    20516,
    46404,
    29479,
    49929,
    30911,
    2605,
    17455,
    23827,
    17038,
    54443,
    48391,
    18175,
    42931,
    40666,
    7794,
    12751,
    39378,
    21440,
    31352,
    32691,
    30371,
    41019,
    36625,
    32695,
    19993,
    2957,
    1220,
    35226,
    51950,
    47614,
    50879,
    29624,
    47900,
    26083,
    31101,
    27241,
    50974,
    34016,
    16929,
    237,
    15057,
    7982,
    43070,
    804,
    33859,
    11306,
    19773,
    28483,
    18325,
]

In [ ]:
# exports


@dataclass
class ActiveLearner:
    acquisition_size: int

    num_validation_samples: int
    num_pool_samples: int

    train_eval_model: TrainEvalModel
    model_trainer: ModelTrainer
    data: ExperimentData

    disable_training_augmentations: bool

    device: Optional

    def __call__(self, log):
        log["seed"] = torch.seed()

        # Active Learning setup
        data = self.data

        train_augmentations = data.train_augmentations if not self.disable_training_augmentations else None

        model_trainer = self.model_trainer
        train_eval_model = self.train_eval_model

        train_loader = model_trainer.get_train_dataloader(data.active_learning.training_dataset)
        pool_loader = model_trainer.get_evaluation_dataloader(data.active_learning.pool_dataset)
        validation_loader = model_trainer.get_evaluation_dataloader(data.validation_dataset)
        test_loader = model_trainer.get_evaluation_dataloader(data.test_dataset)

        log["active_learning_steps"] = []
        active_learning_steps = log["active_learning_steps"]

        # Active Training Loop
        for base_index in predetermind_acquisition_base_indices:
            training_set_size = len(data.active_learning.training_dataset)
            print(f"Training set size {training_set_size}:")

            # iteration_log = dict(training={}, pool_training={}, evaluation_metrics=None, acquisition=None)
            active_learning_steps.append({})
            iteration_log = active_learning_steps[-1]

            iteration_log["training"] = {}

            # TODO: this is a hack! :(
            if data.ood_dataset is None:
                loss = validation_loss = torch.nn.NLLLoss()
            elif data.ood_exposure:
                loss = torch.nn.KLDivLoss(log_target=False, reduction="batchmean")
                validation_loss = torch.nn.NLLLoss()
            else:
                loss = validation_loss = torch.nn.NLLLoss()

            trained_model = model_trainer.get_trained(
                train_loader=train_loader,
                train_augmentations=train_augmentations,
                validation_loader=validation_loader,
                log=iteration_log["training"],
                loss=loss,
                validation_loss=validation_loss,
            )

            evaluation_metrics = evaluate(
                model=trained_model,
                num_samples=self.num_validation_samples,
                loader=test_loader,
                device=self.device,
                storage_device="cpu",
            )
            iteration_log["evaluation_metrics"] = evaluation_metrics
            print(f"Perf after training {evaluation_metrics}")

            iteration_log["acquisition"] = dict(indices=[base_index])
            acquired_label = get_target(data.active_learning.base_dataset, base_index)

            data.active_learning.acquire_base_indices([base_index])

            print(f"Acquiring base index {base_index} {acquired_label}")


@dataclass
class UnifiedExperiment:
    seed: int

    experiment_data_config: ExperimentDataConfig

    acquisition_size: int = 5

    max_training_epochs: int = 300

    num_pool_samples: int = 100
    num_validation_samples: int = 20
    num_training_samples: int = 1

    device: str = "cuda"
    acquisition_function: Union[Type[CandidateBatchComputer], Type[EvalModelBatchComputer]] = acquisition_functions.BALD
    train_eval_model: Type[TrainEvalModel] = TrainSelfDistillationEvalModel
    model_trainer_factory: Type[ModelTrainer] = Cifar10ModelTrainer
    ensemble_size: int = 1

    temperature: float = 0.0
    epig_bootstrap_type: acquisition_functions.BootstrapType = acquisition_functions.BootstrapType.NO_BOOTSTRAP
    epig_bootstrap_factor: float = 1.0
    epig_dtype: torch.dtype = torch.double
    disable_training_augmentations: bool = False
    cache_explicit_eval_model: bool = False

    def load_experiment_data(self) -> ExperimentData:
        print(self.experiment_data_config)
        return self.experiment_data_config.load(self.device)

    # Simple Dependency Injection
    def create_train_eval_model(self) -> TrainEvalModel:
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.train_eval_model)

    def create_model_trainer(self) -> ModelTrainer:
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.model_trainer_factory)

    def run(self, store):
        torch.manual_seed(self.seed)

        # Active Learning setup
        data = self.load_experiment_data()
        store["dataset_info"] = dict(training=repr(data.active_learning.base_dataset), test=repr(data.test_dataset))
        store["initial_training_set_indices"] = data.initial_training_set_indices
        store["evaluation_set_indices"] = data.evaluation_set_indices

        model_trainer = self.create_model_trainer()
        if self.ensemble_size > 1:
            model_trainer = BayesianEnsembleModelTrainer(model_trainer=model_trainer, ensemble_size=self.ensemble_size)
        train_eval_model = self.create_train_eval_model()

        active_learner = ActiveLearner(
            acquisition_size=self.acquisition_size,
            num_validation_samples=self.num_validation_samples,
            num_pool_samples=self.num_pool_samples,
            disable_training_augmentations=self.disable_training_augmentations,
            train_eval_model=train_eval_model,
            model_trainer=model_trainer,
            data=data,
            device=self.device,
        )

        active_learner(store)

## MNIST only

In [ ]:
# exports

# MNIST experiment (ood_exposure=False)

configs = [
    UnifiedExperiment(
        experiment_data_config=StandardExperimentDataConfig(
            id_dataset_name="MNIST",
            id_repetitions=1,
            initial_training_set_size=20,
            validation_set_size=4096,
            validation_split_random_state=0,
            evaluation_set_size=0,
            add_dataset_noise=False,
            ood_dataset_config=None,
        ),
        seed=trial,
        max_training_epochs=120,
        model_trainer_factory=MnistModelTrainer,
        num_pool_samples=100,
        ensemble_size=2,
        device="cuda",
    )
    for trial in range(5)
]

if not is_run_from_ipython() and __name__ == "__main__":
    for job_id, store in embedded_experiments(__file__, len(configs)):
        config = configs[job_id]
        config.seed += job_id
        print(config)
        store["config"] = dataclasses.asdict(config)
        store["log"] = {}

        try:
            config.run(store=store)
        except Exception:
            store["exception"] = traceback.format_exc()
            raise

In [ ]:
results = {}
experiment.max_training_epochs = 1
experiment.run(store=results)

StandardExperimentDataConfig(id_dataset_name='MNIST', id_repetitions=1, initial_training_set_size=20, validation_set_size=4096, validation_split_random_state=0, evaluation_set_size=0, add_dataset_noise=False, ood_dataset_config=None)
Creating: MnistModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=1
)
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=100
)
Training set size 20:


/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

100%|##########| 1/1 [00:00<?, ?it/s]

[1/16]   6%|6          [00:00<?]

[1/32]   3%|3          [00:00<?]

...

[1/16]   6%|6          [00:00<?]

[1/32]   3%|3          [00:00<?]

Epoch metrics: {'accuracy': 0.379150390625, 'crossentropy': 1.8912718296051025}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.379150390625)
RestoringEarlyStopping: Restoring optimizer.


get_predictions_labels:   0%|          | 0/100000 [00:00<?, ?it/s]

KeyboardInterrupt: 